In [2]:
import pandas as pd
import numpy as np
import matplotlib

In [ ]:
# IR_PHA_R

The goal of this notebook is to explore a bit the value table IR_PHA_R.
In particular, we are interested in 4 classes of drugs :
- Antidépresseurs
- Neuroleptiques
- Anxiolytiques
- HTA

For each class, we want to answer different problematics :

- Do we have all the drugs corresponding to the provided description ?
- How can we extract the quantity of each molecule ? Are the unit easy to interpret ?
- How can we see the packaging for each drug ? How much drugs are prescribed for 1 month ? More than 1 month ? Less than 1 month ?
- How many molecules are in this category ?
- Are there molecule from this drug list that are present in drugs outside of this list ?

Also we need to check the consistency between a reference csv version of this value table and the parquet file for each project.

In [1]:
IRPHAR = spark.read.option("header", True).option("delimiter", ";").option("inferSchema", True).csv("D:/firas/fallscripts/ir_pha_r.csv")

let's see the different types of dosages in the table :

In [46]:
print("Number of drugs: ", IRPHAR.count())

Number of drugs:  24143


In [45]:
from pyspark.sql.functions import desc
IRPHAR.groupBy(IRPHAR.PHA_FRM_LIB).count().orderBy(desc("count")).show(30)

+-------------+-----+
|  PHA_FRM_LIB|count|
+-------------+-----+
|     COMPRIME|10357|
|         null| 5246|
|     SOLUTION| 2518|
|       GELULE| 2168|
|       POUDRE| 1163|
|   SUSPENSION|  427|
|      COLLYRE|  357|
|     GRANULES|  211|
|   DISPOSITIF|  201|
| SUPPOSITOIRE|  199|
|        SIROP|  193|
|        CREME|  192|
|          GEL|  171|
|CAPSULE MOLLE|  168|
|      POMMADE|  125|
|        GOMME|   79|
|     EMULSION|   67|
| CAPSULE DURE|   48|
|  LYOPHILISAT|   39|
|      SOLVANT|   35|
|      GRANULE|   25|
|        OVULE|   24|
|         FILM|   19|
|     PASTILLE|   17|
|    COMPRESSE|   16|
|         PATE|   15|
|       LOTION|    9|
|       VERNIS|    7|
|   SHAMPOOING|    6|
|          GAZ|    5|
+-------------+-----+
only showing top 30 rows



##  Drug families

In [4]:
families = ["Antidepresseurs", "Neuroleptiques", "Anxiolyique", "HTA"]
definitions = [["N06A"], ["N05A"], ["N05CD", "N05CF", "N05CX", "N05B", "N05CM11", "N05CM11", "N05CM16"],["C02", "C03", "C07", "C08", "C09", "C10BX04", "C10BX03"]]
ATCStartexceptions = [[], [], [], []]
CIPexceptions = [["3333802"], ["3289633"], [], []]
ATCexceptions = [[], ["N05AL06", "N05AN01"], ["N05CD08"],["C02CA02"]]

## Filter the drug families from IR_PHA_R

In [5]:
from pyspark.sql.functions import udf, count
from pyspark.sql.functions import isnan, count, when, col
test_udf = udf(lambda pattern, ls: True if any(ls).startswith(pattern) else False)

i = 0
AntidepresseursTable = IRPHAR.filter((IRPHAR.PHA_ATC_C07.substr(0,4).alias("ATC").isin(definitions[i])) & \
                                                         (IRPHAR.PHA_ATC_C07.isin(ATCexceptions[i]) == False) & \
                                                         (IRPHAR.PHA_PRS_IDE.isin(CIPexceptions[i]) == False) & \
                                                         (IRPHAR.PHA_ATC_C07.substr(0,5).isin(ATCStartexceptions[i]) == False) )
i = 1
Neuro = IRPHAR.filter((IRPHAR.PHA_ATC_C07.substr(0,4).alias("ATC").isin(definitions[i])) & \
                                                         (IRPHAR.PHA_ATC_C07.isin(ATCexceptions[i]) == False) & \
                                                         (IRPHAR.PHA_PRS_IDE.isin(CIPexceptions[i]) == False) & \
                                                         (IRPHAR.PHA_ATC_C07.substr(0,5).isin(ATCStartexceptions[i]) == False) )

i = 2
Anxio = IRPHAR.filter(((IRPHAR.PHA_ATC_C07.alias("ATC").isin(definitions[i])) | \
                           (IRPHAR.PHA_ATC_C07.substr(0,5).alias("ATC").isin(definitions[i])) | \
                          (IRPHAR.PHA_ATC_C07.substr(0,4).alias("ATC4").isin(definitions[i]))) & \
                                                         (IRPHAR.PHA_ATC_C07.isin(ATCexceptions[i]) == False) & \
                                                         (IRPHAR.PHA_PRS_IDE.isin(CIPexceptions[i]) == False) & \
                                                         (IRPHAR.PHA_ATC_C07.substr(0,5).isin(ATCStartexceptions[i]) == False) )
i = 3
HTA_IRPHAR_ATC = IRPHAR.filter(((IRPHAR.PHA_ATC_C07.substr(0,3).alias("ATC").isin(definitions[i])) | (IRPHAR.PHA_ATC_C07.isin(definitions[i]))) & \
                                                         (IRPHAR.PHA_ATC_C07.isin(ATCexceptions[i]) == False) & \
                                                         (IRPHAR.PHA_PRS_IDE.isin(CIPexceptions[i]) == False) & \
                                                         (IRPHAR.PHA_ATC_C07.substr(0,5).isin(ATCStartexceptions[i]) == False) )
data = AntidepresseursTable.union(Neuro).union(Anxio).union(HTA_IRPHAR_ATC)

In [47]:
data.select("PHA_FRM_LIB").groupBy("PHA_FRM_LIB").count().orderBy(desc("count")).show()
data.select("PHA_DOS_UNT_DSES").distinct().show()

+-------------+-----+
|  PHA_FRM_LIB|count|
+-------------+-----+
|     COMPRIME| 3848|
|         null| 1400|
|       GELULE|  461|
|     SOLUTION|   94|
|CAPSULE MOLLE|   12|
|   SUSPENSION|   11|
|       POUDRE|    7|
|        SIROP|    4|
|  LYOPHILISAT|    2|
+-------------+-----+

+----------------+
|PHA_DOS_UNT_DSES|
+----------------+
|        MG/10 ML|
|           MG/ML|
|         MG/5 ML|
|        G/100 ML|
|       MG/2,5 MG|
|           MG/MG|
|         MG/2 ML|
|              MG|
|          G/5 ML|
|         G/10 ML|
|         G/15 ML|
|               G|
|        MG/MG/MG|
|               %|
|         MG/1 ML|
|         MG/4 ML|
+----------------+



### There are 1400 missing drug forms !! <br>
=> We should investigate the different forms of drugs above and their dosage unity !

In [83]:
columnList = ["PHA_MED_NOM","PHA_NOM_PA", "PHA_CIP_C13", "PHA_ATC_L03", "PHA_FRM_LIB", "PHA_DOS_PRA_DSES", "PHA_DOS_UNT_DSES","PHA_UNT_NBR_DSES", "PHA_CND_TOP"]

In [84]:
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when((data[c].isNull()), c)).alias(c) for c in columnList]).show()

+-----------+----------+-----------+-----------+-----------+----------------+----------------+----------------+-----------+
|PHA_MED_NOM|PHA_NOM_PA|PHA_CIP_C13|PHA_ATC_L03|PHA_FRM_LIB|PHA_DOS_PRA_DSES|PHA_DOS_UNT_DSES|PHA_UNT_NBR_DSES|PHA_CND_TOP|
+-----------+----------+-----------+-----------+-----------+----------------+----------------+----------------+-----------+
|          0|         0|          0|          0|       1400|               0|               0|               0|          0|
+-----------+----------+-----------+-----------+-----------+----------------+----------------+----------------+-----------+



###  missing data is only in PHA_FRM_LIB. <br>
=> Goood news

## dosage unity by form of drug

In [97]:
listForms = ["COMPRIME", "1400", "GELULE", "SOLUTION", "CAPSULE MOLLE", "SUSPENSION", "POUDRE", "SIROP", "LYOPHILISAT"]
for form in listForms:
    df = data.filter(data.PHA_FRM_LIB == form)
    print("Number of ",form, " ",df.count() )
    print("We have the following units :")
    df.groupby("PHA_DOS_UNT_DSES").count().show()
    

Number of  COMPRIME   3848
We have the following units :
+----------------+-----+
|PHA_DOS_UNT_DSES|count|
+----------------+-----+
|       MG/2,5 MG|    1|
|           MG/MG|  911|
|              MG| 2932|
|               G|    2|
|        MG/MG/MG|    2|
+----------------+-----+

Number of  1400   0
We have the following units :
+----------------+-----+
|PHA_DOS_UNT_DSES|count|
+----------------+-----+
+----------------+-----+

Number of  GELULE   461
We have the following units :
+----------------+-----+
|PHA_DOS_UNT_DSES|count|
+----------------+-----+
|           MG/MG|    6|
|              MG|  455|
+----------------+-----+

Number of  SOLUTION   94
We have the following units :
+----------------+-----+
|PHA_DOS_UNT_DSES|count|
+----------------+-----+
|        MG/10 ML|    2|
|           MG/ML|   41|
|         MG/5 ML|    9|
|        G/100 ML|   16|
|         MG/2 ML|    9|
|              MG|    1|
|          G/5 ML|    1|
|         G/10 ML|    1|
|               %|    1|
|     

## Missing data Investigation
We have 1400 drugs without a form

In [93]:
dataMissing = data.filter(data.PHA_FRM_LIB.isNull()).select(columnList)

In [94]:
pd.DataFrame(data = dataMissing.head(5), columns = columnList)

,PHA_MED_NOM,PHA_NOM_PA,PHA_CIP_C13,PHA_ATC_L03,PHA_FRM_LIB,PHA_DOS_PRA_DSES,PHA_DOS_UNT_DSES,PHA_UNT_NBR_DSES,PHA_CND_TOP
0,FLUOXETINE TEVA 20 MG/5 ML 1 FLACON DE 70 ML AVEC,FLUOXETINE,3400936238718,PSYCHOANALEPTIQUES,None,20,MG/5 ML,1/70 ML,NGC
1,"SERTRALINE QUALIMED 50 MG 1 BOITE DE 28, COMPR...",SERTRALINE,3400936942912,PSYCHOANALEPTIQUES,None,50,MG,28,NGC
2,"FLUOXETINE MYLAN 20 MG, 1 BOITE DE 14, GELULES EN",FLUOXETINE,3400935630407,PSYCHOANALEPTIQUES,None,20,MG,14,NGC
3,"PAROXETINE PFIZER 20 MG 1 BOITE DE 28, COMPRIM...",PAROXETINE,3400941503573,PSYCHOANALEPTIQUES,None,20,MG,28,NGC
4,"PAROXETINE EVOLUGEN 20 MG 1 BOITE DE 14, COMPR...",PAROXETINE,3400937427135,PSYCHOANALEPTIQUES,None,20,MG,14,NGC


In [98]:
dataMissing.groupBy("PHA_DOS_UNT_DSES").count().orderBy(desc("count")).show()

+----------------+-----+
|PHA_DOS_UNT_DSES|count|
+----------------+-----+
|              MG| 1128|
|           MG/MG|  252|
|           MG/ML|   10|
|         MG/5 ML|    4|
|         MG/2 ML|    4|
|        MG/MG/MG|    1|
|               %|    1|
+----------------+-----+



In [103]:
df = dataMissing.filter(dataMissing.PHA_DOS_UNT_DSES == "%")
pd.DataFrame(data=df.head(1), columns = columnList)

,PHA_MED_NOM,PHA_NOM_PA,PHA_CIP_C13,PHA_ATC_L03,PHA_FRM_LIB,PHA_DOS_PRA_DSES,PHA_DOS_UNT_DSES,PHA_UNT_NBR_DSES,PHA_CND_TOP
0,"SULPIRIDE GENEVAR 0,5 POUR CENT (SULPIRIDE) 1 ...",SULPIRIDE,3400933651855,PSYCHOLEPTIQUES,None,"0,5",%,1/200 ML,NGC


## Dosage calculus

## Interesting columns

- PHA_NOM_PA  : List of the main molecules contained in the drug 
- PHA_DOS_PRA_DSES (n) :  Quantity of PHA_NOM_PA per drug dose 
- PHA_DOS_UNT_DSES (u) : Unity of a dose 
- PHA_UNT_NBR_DSES (b) : The number of doses contained in the drug 

=> The idea is to use these columns to add a column dosage in mg for each drug. The column dosage contains the overall quantity of each molecule listed in PHA_NOM_PA


In [224]:
mapUnities = {"MG":  1, "G": 1000, "ML": 1, "L": 1000, "%": 0.01}
single_unities = ["MG", "G", "ML", "L"]

def parse_PHA_DOS_UNT_DSES(unity):
    c = unity.count('/')
    parts = unity.split("/",c)
    if(c == 0):
        return mapUnities[parts[0]]
    elif ((c == 1) & (" " in parts[1])):
            (y, b) = parts[1].split(" ",1)
            print(y)
            unity = mapUnities[b]
            y = int(y) * unity
            x = mapUnities[parts[0]]
            return (x, y)
    else:
        mappedParts = map(lambda x: mapUnities[x], parts)
        return  list(mappedParts)


def parse_PHA_UNT_NBR_DSES(qte):
    c = qte.count('/')
    if(c == 0):
        return float(qte)
    else:
        parts = qte.split("/", c)
        x = int(parts[0])
        y = 1
        if(" " in parts[1]):
            (y, b) = parts[1].split(" ")
            unity = mapUnities[b]
            y = int(y) * unity
    return (x, y)
    
def parse_PHA_DOS_PRA_DSES(nbr):
    c = nbr.count('/')
    parts = map(lambda x: x.replace(',', '.'), nbr.split("/",c))
    return list(map(lambda x: float(x), parts))


def split_str(dose, unity, qte):
   
    doses = parse_PHA_DOS_PRA_DSES(dose)
    unities = parse_PHA_DOS_UNT_DSES(unity)
    qtes = parse_PHA_UNT_NBR_DSES(qte)
    print(doses)
    print(unities)
    print(qtes)
    if(unity in single_unities):
        return int(dose) * int(qte) * mapUnities[unity]
    elif(unity == "MG/MG" or unity == "MG/MG/MG"):
        res = list()
        for x,y in zip(doses,unities):
            res.append(x * y * qtes)
        return " ".join(str(x) for x in res)

    elif(unity == "%"):
        return doses[0] * qtes[1] * unities / qtes[0]
    else:
        dose = dose.replace(',', '.')
        (z, w) = parse_PHA_DOS_UNT_DSES(unity)
        (a, b) = parse_PHA_UNT_NBR_DSES(qte)
        return float(dose) * int(z) * int(a) * int(b)/int(w)

dosage_udf = udf(lambda dose, unity, qte : split_str(dose, unity, qte))

In [225]:
listFiltered = dataMissing.select(columnList)
df = listFiltered.withColumn("dosage", dosage_udf(listFiltered.PHA_DOS_PRA_DSES, listFiltered.PHA_DOS_UNT_DSES, listFiltered.PHA_UNT_NBR_DSES))
pd.DataFrame(data = df.head(10), columns = df.columns)

,PHA_MED_NOM,PHA_NOM_PA,PHA_CIP_C13,PHA_ATC_L03,PHA_FRM_LIB,PHA_DOS_PRA_DSES,PHA_DOS_UNT_DSES,PHA_UNT_NBR_DSES,PHA_CND_TOP,dosage
0,FLUOXETINE TEVA 20 MG/5 ML 1 FLACON DE 70 ML AVEC,FLUOXETINE,3400936238718,PSYCHOANALEPTIQUES,None,20,MG/5 ML,1/70 ML,NGC,280.0
1,"SERTRALINE QUALIMED 50 MG 1 BOITE DE 28, COMPR...",SERTRALINE,3400936942912,PSYCHOANALEPTIQUES,None,50,MG,28,NGC,1400
2,"FLUOXETINE MYLAN 20 MG, 1 BOITE DE 14, GELULES EN",FLUOXETINE,3400935630407,PSYCHOANALEPTIQUES,None,20,MG,14,NGC,280
3,"PAROXETINE PFIZER 20 MG 1 BOITE DE 28, COMPRIM...",PAROXETINE,3400941503573,PSYCHOANALEPTIQUES,None,20,MG,28,NGC,560
4,"PAROXETINE EVOLUGEN 20 MG 1 BOITE DE 14, COMPR...",PAROXETINE,3400937427135,PSYCHOANALEPTIQUES,None,20,MG,14,NGC,280
5,"VENLAFAXINE ALMUS 50 MG 1 BOITE DE 30, COMPRIMES",VENLAFAXINE,3400938832532,PSYCHOANALEPTIQUES,None,50,MG,30,NGC,1500
6,"FLUOXETINE RANBAXY 20 MG 1 BOITE DE 28, COMPRIMES",FLUOXETINE,3400938071573,PSYCHOANALEPTIQUES,None,20,MG,28,NGC,560
7,"VENLAFAXINE MYLAN 25 MG 1 BOITE DE 30, COMPRIMES",VENLAFAXINE,3400938534306,PSYCHOANALEPTIQUES,None,25,MG,30,NGC,750
8,"MIANSERINE QUALIMED 10 MG 1 BOITE DE 30, COMPR...",MIANSERINE,3400935766281,PSYCHOANALEPTIQUES,None,10,MG,30,NGC,300
9,"VENLAFAXINE ACTAVIS 50 MG 1 BOITE DE 30, COMPR...",VENLAFAXINE,3400938537666,PSYCHOANALEPTIQUES,None,50,MG,30,NGC,1500


In [112]:
df = dataMissing.filter(dataMissing.PHA_DOS_UNT_DSES == "MG/MG")
pd.DataFrame(data=df.head(1), columns = columnList)

,PHA_MED_NOM,PHA_NOM_PA,PHA_CIP_C13,PHA_ATC_L03,PHA_FRM_LIB,PHA_DOS_PRA_DSES,PHA_DOS_UNT_DSES,PHA_UNT_NBR_DSES,PHA_CND_TOP
0,CAPTOPRIL/HYDROCHLOROTHIAZIDE ZENTIVA 50 MG/25 MG,CAPTOPRIL + HYDROCHLOROTHIAZIDE,3400938370171,MEDICAMENTS AGISSANT SUR LE SYSTEME RENINE-ANG...,None,50/25,MG/MG,90,GC


In [218]:
split_str("50/25", "MG/MG","90")


[50.0, 25.0]
[1, 1]
90.0


'4500.0 2250.0'

In [210]:
split_str("0,5", "%","1/200 ML")


[0.5]
0.01
(1, 200)


1.0

In [211]:
split_str("20", "MG/5 ML", "1/70 ML")

5
[20.0]
(1, 5)
(1, 70)
5


280.0